In [1]:
import numpy as np
import os
import pandas as pd
from datetime import timedelta
import xgboost as xgb
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

In [2]:
def process_dur(data):
    time_difference = pd.to_datetime(data['set_T'],unit='s')-pd.to_datetime(data['rise_T'],unit='s')
    time_difference_in_minutes = time_difference / timedelta(minutes=1)
    data['dur']=time_difference_in_minutes
    return data

In [3]:
def preprocess_xgb(data):
    new_data=data.copy()
    new_data['month'] = new_data.StartTime.str[5:7]
    month_id=['01','02','03','04','05','06','07','08','09','10','11','12']
    month_name=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    sLength = len(new_data.Lse_header)
    new_data['month_name'] = pd.Series(np.random.randn(sLength), index=new_data.index)
    for k in range(0,12):
        new_data.month_name[new_data.month==month_id[k]]=month_name[k]
    new_data=new_data.sort_values(by='StartTime')
    return new_data

In [4]:
def split_test_train_11(new_data,month):
    month_col=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    m_id=month_col.index(month)
    i=m_id
    train=new_data[new_data['month_name']!=month_col[i]]
    train = train.reset_index(drop=True)

    test=new_data[new_data['month_name']==month_col[i]]
    test = test.reset_index(drop=True)

    print('test dimension:')
    print(len(test['High']))
    print('train dimension:')
    print(len(train['High']))
    return train,test

In [5]:
def split_test_train(new_data,month):
    rot_month_id=['01','02','03','04','05','06','07','08','09','10','11','12']
    month_col=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
    m_id=month_col.index(month)
    rot_month_name=['Oct','Nov','Dec','Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec','Jan','Feb','Mar']
    i=m_id+3
    data_1=new_data[new_data['month_name']==rot_month_name[i-1]]
    data_2=new_data[new_data['month_name']==rot_month_name[i-2]]
    data_3=new_data[new_data['month_name']==rot_month_name[i-3]]
    data1=new_data[new_data['month_name']==rot_month_name[i+1]]
    data2=new_data[new_data['month_name']==rot_month_name[i+2]]
    data3=new_data[new_data['month_name']==rot_month_name[i+3]]
    train_frame=[data_3,data_2,data_1,data1,data2,data3]
    train=pd.concat(train_frame,ignore_index=False)
    train = train.reset_index(drop=True)

    test=new_data[new_data['month_name']==rot_month_name[i]]
    #test_frame=[test_data]
    #test=pd.concat(test_frame,ignore_index=True)
    test = test.reset_index(drop=True)


    print('test dimension:')
    print(len(test['High']))
    print('train dimension:')
    print(len(train['High']))
    return train,test

In [6]:
def main(data,month):
    data=process_dur(data)
    new_data=preprocess_xgb(data)
    month='Jan'
    train,test = split_test_train_11(new_data,month)
    return train,test